To compare EasyOCR and Keras OCR on the dataset from Kaggle, you can follow the steps below. This code will load the dataset, apply both OCR models to extract text, and then compare their performance based on accuracy or other metrics like speed, F1 score, etc.

Steps:
Install Dependencies: Make sure you have the required libraries installed.
Load Dataset: Load the images from the Kaggle dataset.
Run OCR Models: Use EasyOCR and Keras OCR on the dataset.
Evaluate Performance: Compare the output using metrics such as accuracy or timing.

# Import necessary libraries


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image

# Load the annotation data from CSV and image data from parquet

In [ ]:
annot = pd.read_csv('/kaggle/input/textocr-text-extraction-from-images-dataset/annot.csv')
imgs = pd.read_parquet('/kaggle/input/textocr-text-extraction-from-images-dataset/img.parquet')

# Load image paths
# Use glob to find all image files in the specified directory
images = glob('/kaggle/input/textocr-text-extraction-from-images-dataset/train_val_images/train_images/*.jpg')  # Corrected to include *.jpg


# Display first few rows of the annotations and images data

In [ ]:
print("Annotations Data:")
print(annot.head())
print("\nImages Data:")
print(imgs.head())

In [ ]:
# Visualize the first image with their corresponding annotations

img_path = images[0]  # Get the image path
image = Image.open(img_path)  # Open the image
plt.imshow(image)  # Display the image
plt.axis('off')  # Hide axes
plt.title(f"Image: {img_path.split('/')[-1]}")  # Title as the image name
plt.show()

In [ ]:
image_id = images[0].split('/')[-1].split('.')[0] 
annot.query('image_id == @image_id')

In [ ]:
# Display the first 9 images in a 3x3 grid
for i in range(9):  # Loop through the first 9 images
    img_path = images[i]  # Get the image path
    image = Image.open(img_path)  # Open the image
    
    plt.subplot(3, 3, i + 1)  # Create a subplot in a 3x3 grid
    plt.imshow(image)  # Display the image
    plt.axis('off')  # Hide axes
    plt.title(f"Image: {img_path.split('/')[-1]}",fontsize=8)  # Title as the image name

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


# Initialize OCR models


In [ ]:
# Initialize OCR models
# please install both this if u dont have installed

!pip install keras-ocr tensorflow==2.12.0
!pip install numpy matplotlib
import keras_ocr
import easyocr
pipeline = keras_ocr.pipeline.Pipeline()  # Keras OCR
reader = easyocr.Reader(['en'], gpu=True)  # EasyOCR

# Function to process OCR results


In [ ]:
# Function to process EasyOCR results
def get_easyocr_results(image_paths):
    dfs = []
    for img in tqdm(image_paths):
        result = reader.readtext(img)
        img_id = img.split('/')[-1].split('.')[0]
        img_df = pd.DataFrame(result, columns=['bbox', 'text', 'conf'])
        img_df['img_id'] = img_id
        dfs.append(img_df)
    return pd.concat(dfs)

# Function to process Keras OCR results
def get_kerasocr_results(image_paths):
    dfs = []
    for img in tqdm(image_paths):
        results = pipeline.recognize([img])
        img_id = img.split('/')[-1].split('.')[0]
        img_df = pd.DataFrame(results[0], columns=['text', 'bbox'])
        img_df['img_id'] = img_id
        dfs.append(img_df)
    return pd.concat(dfs)


# Get results for EasyOCR and Keras OCR



In [ ]:
# Get results for EasyOCR and Keras OCR
easyocr_df = get_easyocr_results(images[:25])  # Use first 25 images for EasyOCR
kerasocr_df = get_kerasocr_results(images[:25])  # Use first 25 images for Keras OCR


# Function to plot comparison between EasyOCR and Keras OCR results


In [ ]:
def plot_compare(img_fn, easyocr_df, kerasocr_df):
    img_id = img_fn.split('/')[-1].split('.')[0]  # Extract the image ID
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))

    # EasyOCR results for the current image
    easy_results = easyocr_df.query('img_id == @img_id')[['text', 'bbox']].values.tolist()
    easy_results = [(x[0], np.array(x[1])) for x in easy_results]  # Format as (text, bbox) pairs

    # Visualize EasyOCR results
    keras_ocr.tools.drawAnnotations(plt.imread(img_fn), easy_results, ax=axs[0])
    axs[0].set_title('EasyOCR Results', fontsize=24)

    # Keras OCR results for the current image
    keras_results = kerasocr_df.query('img_id == @img_id')[['text', 'bbox']].values.tolist()
    keras_results = [(x[0], np.array(x[1])) for x in keras_results]  # Format as (text, bbox) pairs

    # Visualize Keras OCR results
    keras_ocr.tools.drawAnnotations(plt.imread(img_fn), keras_results, ax=axs[1])
    axs[1].set_title('Keras OCR Results', fontsize=24)

    # Show the comparison plot
    plt.show()

# Example usage: visualize OCR results for the first image


In [ ]:
for img_path in images[:20]:  # Adjust the range as needed
    plot_compare(img_path, easyocr_df, kerasocr_df)